# Tutorial: Adding User Feedback to LLM Applications with Phoenix

This tutorial demonstrates how to capture and analyze user feedback for LLM applications using Phoenix's tracing and annotation capabilities. We'll build a social media post generator that allows users to provide feedback on generated content.

## Overview
1. Set up Phoenix and OpenAI
2. Create a post generator function
3. Implement feedback mechanism
4. Test the system
5. Analyze feedback in Phoenix

## 1. Install Dependencies

In [ ]:
!pip install -q arize-phoenix openai openinference-instrumentation-openai 'httpx<0.28'

## 2. Configure Phoenix and OpenAI

In [4]:
# Import required packages
import os
from openai import OpenAI
import httpx
from phoenix.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor
from opentelemetry import trace
from openinference.semconv.trace import SpanAttributes

# Configure Phoenix
if "PHOENIX_API_KEY" in os.environ:
    # If using cloud instance of Phoenix
    os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
else:
    # Locally start an instance
    import phoenix as px
    px.launch_app().view()

# Register tracer
tracer_provider = register(project_name="social-media-post-generator")
trace.set_tracer_provider(tracer_provider)
tracer = trace.get_tracer(__name__)

# Instrument OpenAI
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

# Initialize OpenAI client
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    from getpass import getpass
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

client = OpenAI()

Overriding of current TracerProvider is not allowed


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
📺 Opening a view to the Phoenix app. The app is running at http://localhost:6006/
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: social-media-post-generator
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



🔑 Enter your OpenAI API key:  ········


## 3. Define Post Generator and Feedback Functions

In [7]:
PROMPT_TEMPLATE = """
You are an expert social media content creator.
Your task is to create a different promotion message with the following 
Product Description :
------
{product_desc}
------
The output promotion message should have the following :
Title: a powerful, short message that dipict what this product is about 
Message: be creative for the promotion message, but make it short and ready for social media feeds, under 100 words.
Tags: the hash tag human will nomally use in social media

Give me the final post, do not have "Title:", "Message:", "Tags:" in it.
Begin!
"""

def generate_post(description):
    """Generate a social media post with tracing"""
    try:
        attributes = {SpanAttributes.OPENINFERENCE_SPAN_KIND: "CHAIN"}
        with tracer.start_as_current_span("Social media post", attributes=attributes) as span:
            span.set_attribute(SpanAttributes.INPUT_VALUE, description)
            
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a helpful social media content creator."},
                    {"role": "user", "content": PROMPT_TEMPLATE.format(product_desc=description)}
                ]
            )
            
            output_content = response.choices[0].message.content
            span.set_attribute(SpanAttributes.OUTPUT_VALUE, output_content)
            
            span_context = span.get_span_context()
            span_id = format(span_context.span_id, '016x')
            
            return output_content, span_id
    except Exception as e:
        return f"Error generating post: {str(e)}", None
    

def send_feedback_to_phoenix(span_id, feedback_type):
    """Send feedback annotation to Phoenix"""
    if not span_id:
        return False
        
    client = httpx.Client()
    label = "👍" if feedback_type == "like" else "👎"
    score = 1 if feedback_type == "like" else 0
    
    try:
        annotation_payload = {
            "data": [
                {
                    "span_id": span_id,
                    "name": "user feedback",
                    "annotator_kind": "HUMAN",
                    "result": {
                        "label": label, 
                        "score": score,
                        "explanation": f"User provided {feedback_type} feedback"
                    },
                    "metadata": {}
                }
            ]
        }

        response = client.post(
            "http://localhost:6006/v1/span_annotations?sync=false",
            json=annotation_payload
        )
        return response.status_code == 200
    except Exception:
        return False

## 4. Test the System

Let's test our post generator and feedback system with a sample product description.

In [8]:
# Sample product description
product_description = """
EcoFresh Water Bottle - A sustainable, insulated water bottle made from recycled materials.
Keeps drinks cold for 24 hours or hot for 12 hours. Available in various colors.
Features a leak-proof lid and carries 32oz of your favorite beverage.
"""

# Generate post
generated_post, span_id = generate_post(product_description)
print("Generated Post:\n", generated_post)
print("\nSpan ID:", span_id)

Generated Post:
 Sip Sustainably, Live Refreshingly!

Discover the EcoFresh Water Bottle: your new eco-friendly companion that keeps every sip perfectly cold for 24 hours or piping hot for 12. Crafted from recycled materials, this 32oz marvel features a sleek, leak-proof design in your favorite hues. Stay refreshed and reduce waste with every gulp. Eco-convenience never looked so good!

#EcoFresh #HydrateResponsibly #SustainableLiving #StayCool #EcoFriendlyLiving #DrinkSmart #EarthLovers

Span ID: 27df1f6642d34166


## 5. Send Feedback

Now let's simulate user feedback for the generated post.

In [9]:
# Send positive feedback
success = send_feedback_to_phoenix(span_id, "like")
print(f"Feedback sent successfully: {success}")

# You can also try negative feedback
# success = send_feedback_to_phoenix(span_id, "dislike")
# print(f"Feedback sent successfully: {success}")

Feedback sent successfully: True


## 6. Analyzing Feedback in Phoenix

Now that we've generated content and collected feedback, you can view the results in Phoenix:

1. Open your Phoenix dashboard
2. Navigate to the Traces view
3. Look for traces with the name "Social media post"
4. Click on a trace to see details including:
   - Input product description
   - Generated content
   - User feedback (👍 or 👎)
   - Timing information

This feedback data can help you:
- Identify patterns in well-received vs poorly-received content
- Monitor the quality of generated posts
- Make data-driven improvements to your prompt engineering
- Track user satisfaction over time